# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset from the command line with:

`mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`

Note that the path runs from the repo root directory, the parent of the 'Resources' folder.

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
thing = mongo.list_database_names()

print("The extant databases:\n")
print("\n".join(thing))
print("\nDatabase import worked if uk_food is included")

The extant databases:

admin
config
local
met
uk_food

Database import worked if uk_food is included


In [4]:
# assign the uk_food database to a variable name
db = mongo.uk_food

In [5]:
# review the collections in our new database
# * to unpack, output is more reader-friendly
print("Collections in the uk_food database:\n")
print(*db.list_collection_names())

Collections in the uk_food database:

establishments


In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 551803,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64f15b358a05d41a906f6844'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/551803',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate':

In [7]:
# assign the collection to a variable
establishments = db.establishments

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {"Hygiene": "", "Structural": "", "ConfidenceInManagement": ""},
    "SchemeType": "FHRS",
    "geocode": {"longitude": "0.08384000", "latitude": "51.49014200"},
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True,
}

In [9]:
# Insert the new restaurant into the collection
# formatting - the semicolon suppresses output, reducing clutter, a trick frequently used in the ensuing cells
establishments.insert_one(new_restaurant);

In [10]:
# Check that the new restaurant was inserted
q = {"BusinessName": "Penang Flavours"}
pprint(establishments.find_one(q))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64f15b42e9afe0bd977fbd7e'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
q = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = ["BusinessTypeID", "BusinessType"]

_ = establishments.find_one(q, fields)
print(f"BusinessType: {_['BusinessType']}\nBusinessTypeID: {_['BusinessTypeID']}")

BusinessType: Restaurant/Cafe/Canteen
BusinessTypeID: 1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set": {"BusinessTypeID": "1"}});

In [13]:
# Confirm that the new restaurant was updated
q = {"BusinessName": "Penang Flavours"}
_ = establishments.find_one(q, ["BusinessName", "BusinessTypeID"])
print(f"{_['BusinessName']} now has a Business Type ID of {_['BusinessTypeID']}\n")

Penang Flavours now has a Business Type ID of 1



4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
q = {"LocalAuthorityName": "Dover"}
print(f"The number of documents from the Dover Local Authority is {establishments.count_documents(q)}")
print(f"The total number of documents is {establishments.count_documents({})}")

The number of documents from the Dover Local Authority is 994
The total number of documents is 39780


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"});

In [16]:
# Check if any remaining documents include Dover
q = {"LocalAuthorityName": "Dover"}
print(f"The number of documents from the Dover Local Authority is {establishments.count_documents(q)}")
print(f"The total number of documents is {establishments.count_documents({})}")

The number of documents from the Dover Local Authority is 0
The total number of documents is 38786


In [17]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('64f15b368a05d41a906f6b2a'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}])
establishments.update_many({}, [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]);

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = [
    "AwaitingInspection",
    "Awaiting Inspection",
    "AwaitingPublication",
    "Pass",
    "Exempt",
]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [{"$set": {"RatingValue": None}}]);

In [20]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]);

In [21]:
# Check that the coordinates and rating value are now numbers
q = {}
fields = ["geocode.latitude", "geocode.longitude", "RatingValue"]
d_t = data_types = establishments.find_one(q, fields)

# print data types
print(
    f"""Longitude data type: {type(d_t['geocode']['longitude'])}
Latitude data type: {type(d_t['geocode']['latitude'])}
RatingValue data type: {type(d_t['RatingValue'])}
"""
)

Longitude data type: <class 'float'>
Latitude data type: <class 'float'>
RatingValue data type: <class 'int'>

